
# Casos de Uso de Machine Learning – Energia Solar e Eólica

Este notebook contém a resolução das tarefas propostas na disciplina **Soluções em Energias Renováveis e Sustentáveis**.  O objetivo é aplicar algoritmos de regressão e classificação a diferentes conjuntos de dados relacionados a energia, comparando modelos e discutindo os resultados obtidos.

As atividades estão divididas em quatro blocos:

1. **Parte 1 – Regressão (Appliances Energy Prediction)**: prever o consumo de eletrodomésticos a partir de variáveis ambientais.
2. **Parte 2 – Classificação (Smart Grid Stability)**: classificar se a rede elétrica está estável ou instável com base em medições elétricas.
3. **Exercício 1 – Classificação (Solar)**: classificar períodos de alta ou baixa radiação solar a partir de dados meteorológicos.
4. **Exercício 2 – Regressão (Eólica)**: prever a potência gerada por uma turbina eólica utilizando medidas de vento e potência teórica.

> **Nota:** sempre que os arquivos CSV reais não estiverem disponíveis, dados sintéticos são gerados para fins de demonstração.  Para reproduzir os resultados com os datasets originais, copie os arquivos correspondentes para a pasta `data/` indicada no README e ajuste os nomes se necessário.


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

sns.set(style="whitegrid")


In [ ]:

# Funções auxiliares para avaliação e carregamento de dados

def evaluate_regression(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    return {"R²": r2, "RMSE": rmse, "MAE": mae}


def evaluate_classification(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    pos_label = y_test.unique()[0] if hasattr(y_test, 'unique') else 1
    prec = precision_score(y_test, preds, average='binary', pos_label=pos_label)
    rec = recall_score(y_test, preds, average='binary', pos_label=pos_label)
    f1 = f1_score(y_test, preds, average='binary', pos_label=pos_label)
    cm = confusion_matrix(y_test, preds)
    return {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1": f1, "Confusion Matrix": cm}


def load_dataset(path, synthetic_generator):
    if os.path.exists(path):
        df = pd.read_csv(path)
        return df, False
    else:
        print(f"Arquivo {path} não encontrado. Usando dados sintéticos para demonstração.")
        X, y = synthetic_generator()
        return pd.concat([X, y], axis=1), True


In [ ]:

# Geradores de dados sintéticos

def synthetic_appliances_data(n_samples=2000, random_state=42):
    rng = np.random.RandomState(random_state)
    temps = rng.uniform(10, 30, size=(n_samples, 5))
    humidity = rng.uniform(20, 90, size=(n_samples, 5))
    pressure = rng.uniform(990, 1030, size=(n_samples, 1))
    windspeed = rng.uniform(0, 10, size=(n_samples, 1))
    coeffs = np.array([3.5, -1.2, 4.0, 2.1, -0.8] + [-0.5, 0.3, -1.1, 0.7, 0.5] + [0.02] + [-1.5])
    features = np.hstack([temps, humidity, pressure, windspeed])
    noise = rng.normal(0, 5, size=n_samples)
    y = features.dot(coeffs) + noise
    cols = [f"temp_{i+1}" for i in range(5)] + [f"hum_{i+1}" for i in range(5)] + ["pressure", "windspeed"]
    X = pd.DataFrame(features, columns=cols)
    y = pd.Series(y, name="Appliances")
    return X, y


def synthetic_smart_grid_data(n_samples=2000, random_state=42):
    rng = np.random.RandomState(random_state)
    features = rng.uniform(-1, 1, size=(n_samples, 4))
    linear = features.dot(np.array([1.0, -1.2, 0.8, -0.5])) + rng.normal(0, 0.2, size=n_samples)
    y = (linear > 0).astype(int)
    X = pd.DataFrame(features, columns=["P_active", "P_reactive", "Voltage", "Current"])
    y = pd.Series(y, name="Stability")
    return X, y


def synthetic_solar_data(n_samples=1500, random_state=42):
    rng = np.random.RandomState(random_state)
    temperature = rng.uniform(15, 40, n_samples)
    humidity = rng.uniform(10, 90, n_samples)
    pressure = rng.uniform(950, 1030, n_samples)
    wind = rng.uniform(0, 10, n_samples)
    clouds = rng.uniform(0, 100, n_samples)
    radiation = 0.02 * temperature - 0.01 * humidity + 0.03 * wind + rng.normal(0, 0.5, n_samples)
    median_rad = np.median(radiation)
    y = (radiation >= median_rad).astype(int)
    X = pd.DataFrame({
        "Temperature": temperature,
        "Humidity": humidity,
        "Pressure": pressure,
        "WindSpeed": wind,
        "CloudCoverage": clouds,
    })
    y = pd.Series(y, name="HighRadiation")
    return X, y


def synthetic_wind_data(n_samples=1500, random_state=42):
    rng = np.random.RandomState(random_state)
    wind_speed = rng.uniform(0, 25, n_samples)
    wind_direction = rng.uniform(0, 360, n_samples)
    theoretical_power = 0.5 * wind_speed ** 3
    efficiency = rng.uniform(0.2, 0.45, n_samples)
    active_power = theoretical_power * efficiency + rng.normal(0, 50, n_samples)
    X = pd.DataFrame({
        "WindSpeed": wind_speed,
        "WindDirection": wind_direction,
        "TheoreticalPower": theoretical_power,
    })
    y = pd.Series(active_power, name="LVActivePower")
    return X, y



## Parte 1 – Regressão: Previsão de consumo de eletrodomésticos

O objetivo desta seção é prever o consumo de energia (Wh) de uma residência a partir de variáveis ambientais coletadas ao longo de 4,5 meses.  Os dados originais, descritos no [repositório da UCI](https://archive.ics.uci.edu/dataset/374/appliances+energy+prediction), incluem temperatura, umidade e condições climáticas externas registradas em intervalos de 10 minutos 【722290714644327†L46-L54】.  Como os valores nem sempre estão disponíveis em todas as máquinas, o código abaixo tenta carregar o arquivo `energydata_complete.csv` na pasta `data/`; caso não exista, um conjunto sintético é gerado para demonstração.


In [ ]:

# Parte 1: carregar dataset de appliances
df_appliances, synthetic_flag = load_dataset('data/energydata_complete.csv', synthetic_appliances_data)

# Preparar features e alvo
if not synthetic_flag:
    df_appliances['date'] = pd.to_datetime(df_appliances['date'])
    df_appliances['hour'] = df_appliances['date'].dt.hour
    df_appliances['dayofweek'] = df_appliances['date'].dt.dayofweek
    feature_cols = [col for col in df_appliances.columns if col not in ['date', 'Appliances']]
    X_appliances = df_appliances[feature_cols]
    y_appliances = df_appliances['Appliances']
else:
    X_appliances = df_appliances.drop(columns=['Appliances'])
    y_appliances = df_appliances['Appliances']

# Dividir em treino e teste
X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(X_appliances, y_appliances, test_size=0.3, random_state=42)

# Normalizar
scaler_a = StandardScaler()
X_train_a_scaled = scaler_a.fit_transform(X_train_a)
X_test_a_scaled = scaler_a.transform(X_test_a)

# Definir modelos
regressors = {
    'Regressão Linear': LinearRegression(),
    'Árvore de Regressão': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=200)
}

results_reg = {}
for name, model in regressors.items():
    metrics = evaluate_regression(model, X_train_a_scaled, y_train_a, X_test_a_scaled, y_test_a)
    results_reg[name] = metrics

# Mostrar resultados
pd.DataFrame(results_reg).T



**Discussão:**

O quadro acima resume o desempenho de três modelos de regressão.  A Regressão Linear serve como baseline e assume uma relação linear entre as variáveis ambientais e o consumo de energia.  A Árvore de Regressão captura relações não lineares simples, enquanto a Random Forest, composta por várias árvores, tende a modelar interações complexas e reduzir o overfitting.  Em geral, espera‑se que a Random Forest apresente maior R² e menor RMSE/MAE devido à sua capacidade de generalizar melhor.  Entretanto, a diferença real só pode ser avaliada com os dados originais; os dados sintéticos aqui servem apenas para ilustrar a metodologia.



## Parte 2 – Classificação: Estabilidade de rede elétrica inteligente

Nesta seção utilizamos um dataset de estabilidade de rede (não público) para treinar modelos que classifiquem se a rede está **estável** ou **instável** a partir de medidas elétricas como potência ativa, potência reativa, tensão e corrente.  Como os dados podem não estar disponíveis, o código tenta ler `smart_grid_stability.csv` na pasta `data/` e, caso não exista, gera um dataset sintético.  Os algoritmos avaliados são uma Árvore de Decisão, KNN e Regressão Logística.


In [ ]:

# Parte 2: carregar dataset de estabilidade
df_grid, synthetic_flag_grid = load_dataset('data/smart_grid_stability.csv', synthetic_smart_grid_data)

if not synthetic_flag_grid:
    target_col = [col for col in df_grid.columns if col.lower().startswith('stab')][0]
    X_grid = df_grid.drop(columns=[target_col])
    y_grid = df_grid[target_col]
else:
    X_grid = df_grid.drop(columns=['Stability'])
    y_grid = df_grid['Stability']

X_train_g, X_test_g, y_train_g, y_test_g = train_test_split(X_grid, y_grid, test_size=0.3, random_state=42, stratify=y_grid)

scaler_g = StandardScaler()
X_train_g_scaled = scaler_g.fit_transform(X_train_g)
X_test_g_scaled = scaler_g.transform(X_test_g)

classifiers = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'KNN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'Regressão Logística': LogisticRegression(max_iter=1000)
}

results_clf = {}
for name, model in classifiers.items():
    metrics = evaluate_classification(model, X_train_g_scaled, y_train_g, X_test_g_scaled, y_test_g)
    results_clf[name] = {k: v for k, v in metrics.items() if k != 'Confusion Matrix'}
    results_clf[name]['Confusion Matrix'] = metrics['Confusion Matrix']

# Mostrar resultados sem matriz de confusão
pd.DataFrame({k: {metric: v for metric, v in m.items() if metric != 'Confusion Matrix'} for k, m in results_clf.items()}).T


In [ ]:

# Matrizes de confusão para cada modelo de classificação
title_cmap = {'Árvore de Decisão': 'Blues', 'KNN (k=5)': 'Blues', 'Regressão Logística': 'Blues'}
for name, metrics in results_clf.items():
    cm = metrics['Confusion Matrix']
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap=title_cmap[name])
    plt.title(f"Matriz de Confusão – {name}")
    plt.xlabel("Predito")
    plt.ylabel("Verdadeiro")
    plt.show()



**Discussão:**

A comparação entre os classificadores mostra como diferentes algoritmos tratam a separação entre classes.  A Árvore de Decisão é fácil de interpretar, mas pode sofrer overfitting; o KNN depende da escolha de `k` e da distância entre pontos; a Regressão Logística pressupõe uma relação linear entre as variáveis e a logit da probabilidade.  A seleção do modelo mais confiável deve considerar acurácia, recall e F1‑score, pois cada métrica enfatiza um aspecto (taxa de acertos gerais, sensibilidade a falsos negativos e equilíbrio entre precisão e recall, respectivamente).



## Exercício 1 – Classificação: Previsão de nível de radiação solar

Aqui utilizamos o dataset de radiação solar para classificar períodos em **Alta Radiação** e **Baixa Radiação**.  A variável alvo é criada com base na mediana da radiação solar.  Caso o arquivo `SolarPrediction.csv` (ou similar) não esteja disponível, serão gerados dados sintéticos.  Os algoritmos avaliados são Árvore de Decisão, Random Forest e Support Vector Machine (SVM).


In [ ]:

# Exercício 1: carregar dataset de radiação solar
df_solar, synthetic_flag_solar = load_dataset('data/SolarPrediction.csv', synthetic_solar_data)

if not synthetic_flag_solar:
    rad_col = [c for c in df_solar.columns if 'radiat' in c.lower() or 'solar' in c.lower()][0]
    median_rad = df_solar[rad_col].median()
    df_solar['HighRadiation'] = (df_solar[rad_col] >= median_rad).astype(int)
    feature_cols = [col for col in df_solar.columns if col not in [rad_col, 'HighRadiation']]
    X_solar = df_solar[feature_cols]
    y_solar = df_solar['HighRadiation']
else:
    X_solar = df_solar.drop(columns=['HighRadiation'])
    y_solar = df_solar['HighRadiation']

X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_solar, y_solar, test_size=0.3, random_state=42, stratify=y_solar)

scaler_s = StandardScaler()
X_train_s_scaled = scaler_s.fit_transform(X_train_s)
X_test_s_scaled = scaler_s.transform(X_test_s)

classifiers_solar = {
    'Árvore de Decisão': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=200),
    'SVM (linear)': SVC(kernel='linear')
}

results_solar = {}
for name, model in classifiers_solar.items():
    metrics = evaluate_classification(model, X_train_s_scaled, y_train_s, X_test_s_scaled, y_test_s)
    results_solar[name] = {k: v for k, v in metrics.items() if k != 'Confusion Matrix'}
    results_solar[name]['Confusion Matrix'] = metrics['Confusion Matrix']

# Mostrar resultados
pd.DataFrame({k: {metric: v for metric, v in m.items() if metric != 'Confusion Matrix'} for k, m in results_solar.items()}).T


In [ ]:

# Matrizes de confusão para a classificação de radiação solar
for name, metrics in results_solar.items():
    cm = metrics['Confusion Matrix']
    plt.figure(figsize=(4,3))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens')
    plt.title(f"Matriz de Confusão – {name}")
    plt.xlabel("Predito")
    plt.ylabel("Verdadeiro")
    plt.show()



**Discussão:**

A SVM geralmente se destaca em problemas de classificação binária com fronteira bem definida, mas pode exigir mais ajustes de parâmetros (por exemplo, escolha do kernel e valor de `C`).  A Random Forest tende a apresentar boa acurácia e robustez, pois combina diversas árvores.  A Árvore de Decisão é simples e interpretável, embora possa ter desempenho inferior se houver muitas variáveis correlacionadas.  A escolha do melhor modelo deve levar em conta a acurácia, a matriz de confusão e o custo de falsos positivos/negativos no contexto de gestão de sistemas fotovoltaicos.



## Exercício 2 – Regressão: Previsão de potência de turbinas eólicas

Esta seção utiliza dados de SCADA de turbinas eólicas para prever a potência ativa gerada (kW) a partir de variáveis como velocidade do vento, direção e potência teórica.  Se o arquivo `wind_scada.csv` não estiver disponível, dados sintéticos serão gerados.  Os algoritmos avaliados são Regressão Linear, Árvore de Regressão e Random Forest Regressor.


In [ ]:

# Exercício 2: carregar dataset de turbinas eólicas
df_wind, synthetic_flag_wind = load_dataset('data/wind_scada.csv', synthetic_wind_data)

if not synthetic_flag_wind:
    target_col = [c for c in df_wind.columns if 'active' in c.lower() or ('power' in c.lower() and 'theoretical' not in c.lower())][0]
    feature_cols = [col for col in df_wind.columns if col != target_col]
    X_wind = df_wind[feature_cols]
    y_wind = df_wind[target_col]
else:
    X_wind = df_wind.drop(columns=['LVActivePower'])
    y_wind = df_wind['LVActivePower']

X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_wind, y_wind, test_size=0.2, random_state=42)

scaler_w = StandardScaler()
X_train_w_scaled = scaler_w.fit_transform(X_train_w)
X_test_w_scaled = scaler_w.transform(X_test_w)

regressors_wind = {
    'Regressão Linear': LinearRegression(),
    'Árvore de Regressão': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=200)
}

results_wind = {}
for name, model in regressors_wind.items():
    metrics = evaluate_regression(model, X_train_w_scaled, y_train_w, X_test_w_scaled, y_test_w)
    results_wind[name] = metrics

pd.DataFrame(results_wind).T



**Discussão:**

A previsão de potência de turbinas eólicas envolve relações não lineares entre a velocidade do vento e a potência gerada.  Embora a Regressão Linear forneça um baseline, espera‑se que modelos não lineares (Árvore de Regressão e Random Forest) captem melhor a forma cúbica da curva potência × velocidade.  A Random Forest costuma oferecer melhor generalização e menor erro em comparação a uma única árvore, especialmente quando há ruído nos dados (como oscilações de vento e variações de eficiência da turbina).
